In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
from numpy import exp
import matplotlib.pyplot as plt
import seaborn as sns
from math import sqrt, log, atan2
from scipy.special import sph_jn, sph_yn

In [ ]:
sns.set_style('darkgrid')

In [ ]:
mu_hbar_const = 0.0478450  # 2mu/hbar^2

In [ ]:
def hanplus(rho, L):
    return -rho * sph_yn(L, rho)[0][-1] + (rho * sph_jn(L, rho)[0][-1])*1j
def hanminus(rho, L):
    return -rho * sph_yn(L, rho)[0][-1] - (rho * sph_jn(L, rho)[0][-1])*1j
def hanplus_prime(rho, L, k):
    j = sph_jn(L, rho)[0][-1]
    jp = sph_jn(L, rho)[1][-1]
    y = sph_yn(L, rho)[0][-1]
    yp = sph_yn(L, rho)[1][-1]
    return -k * (y + 1j*j + rho * (yp + 1j*jp))
def hanminus_prime(rho, L, k):
    j = sph_jn(L, rho)[0][-1]
    jp = sph_jn(L, rho)[1][-1]
    y = sph_yn(L, rho)[0][-1]
    yp = sph_yn(L, rho)[1][-1]
    return -k * (y - 1j*j + rho * (yp - 1j*jp))

In [ ]:
def pot(r):
    """The Woods-Saxon potential.
    
    Parameters
    ----------
    r : number
        The radius, in fm.
    
    Returns
    -------
    number
        The potential at r, in MeV
    """
    A = 10
#     A = 2000
    v0 = -61.1  # MeV
    aws = 0.65  # fm
    rws = 1.2 * A**(1/3)  # fm
    return v0 / (1 + exp((r-rws)/aws))

In [ ]:
def diffeq(x, r, L, E):
    """Differential equation for scattering.
    
    Parameters
    ----------
    x : array-like
        The vector of [u, u'], where u is the solution to the diff eq.
    r : number
        The radial position, in fm.
    L : integer
        The angular momentum quantum number
    E : number
        The energy, in MeV.
        
    Returns
    -------
    ndarray
        The vector of derivatives [u', u'']
    """
    x1, x2 = x
    x1p = x2
    try:
        x2p = ((L * (L+1))/r**2 + 0.0478450 * (pot(r) - E)) * x1
    except ZeroDivisionError:
        if x1 == 0:
            x2p = 0.
        else:
            x2p = 1e9
    return np.array((x1p, x2p))

In [ ]:
def RK4(x, func, r, dr, *args):
    """Fourth-order Runge-Kutta integrator.
    
    Parameters
    ----------
    x : array-like, or number
        The state vector, or current state
    func : callable
        The differential equation. Its call signature should be func(x, r, ...). 
        Any additional arguments to RK4 will be passed on to func.
    r : number
        The current radial position coordinate, in fm.
    dr : number
        The position step, in fm.
    
    Returns
    -------
    xnew : ndarray or number
        The next state vector or state
    """
    rka = func(x, r, *args)
    rkb = func(x + (dr / 2) * rka, r + dr/2, *args)
    rkc = func(x + (dr / 2) * rkb, r + dr/2, *args)
    rkd = func(x + (dr * rkc), r + dr, *args)

    xnew = x + (dr / 6) * (rka + 2 * rkb + 2 * rkc + rkd)

    return xnew

In [ ]:
def integrate_u(energy, angmom):
    x0 = np.array((0, 1e-3))

    npts = 10000
    r0 = 0
    dr = 0.01

    xpts = np.arange(npts) * dr + r0
    pts = np.zeros((npts, 2))

    x = x0
    pts[0, :] = x

    for i in range(1, npts):
        x = RK4(x, diffeq, r0 + dr*i, dr, angmom, energy)
        pts[i, :] = x
    
    return xpts, pts

In [ ]:
xpts, pts = integrate_u(3, 1)
plt.plot(xpts, pts[:, 0])
res_scale = pts[:, 0].min()
plt.plot(xpts, [pot(x) / pot(0) * res_scale for x in xpts])

In [ ]:
def find_phase(xpts, pts, energy, angmom):
    rmat = (pts[-1, 0] / pts[-1, 1]) / xpts[-1]  # rmatrix
    k = sqrt(mu_hbar_const * energy)
    rho = k * xpts[-1]
    numer = hanminus(rho, angmom) - xpts[-1]*rmat*hanminus_prime(rho, angmom, k)
    denom = hanplus(rho, angmom) - xpts[-1]*rmat*hanplus_prime(rho, angmom, k)
    smat = numer / denom
    assert sqrt(smat.imag**2 + smat.real**2) - 1.0 < 0.1, 'smat magnitude not 1'
    return 0.5 * atan2(smat.imag, smat.real)

In [ ]:
find_phase(xpts, pts, energy, angmom)